# Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Import the necessary libs

import os

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool

In [2]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [3]:
# Create retrieve_game tool

import chromadb
from chromadb.utils import embedding_functions

@tool
def retrieve_game(query: str) -> str:
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    try:
        # Connect to the persistent Chroma database used in Part 01
        chroma_client = chromadb.PersistentClient(path="chromadb")

        # Ensure we use the same embedding function configuration as when the collection was created
        embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
            api_key=OPENAI_API_KEY
        )

        # Get the existing collection
        collection = chroma_client.get_collection(
            name="udaplay",
            embedding_function=embedding_fn
        )

        # Perform semantic search over stored documents
        results = collection.query(
            query_texts=[query],
            n_results=5,
            include=["documents", "metadatas", "distances"],
        )

        formatted_results = []
        docs = results.get("documents") or []
        metas = results.get("metadatas") or []
        dists = results.get("distances") or []

        if docs and len(docs) > 0:
            for i, doc in enumerate(docs[0]):
                meta = metas[0][i] if metas and metas[0] and i < len(metas[0]) else {}
                dist = dists[0][i] if dists and dists[0] and i < len(dists[0]) else None

                description = meta.get("Description") or doc
                formatted_results.append({
                    "Platform": meta.get("Platform", "Unknown"),
                    "Name": meta.get("Name", "Unknown"),
                    "YearOfRelease": meta.get("YearOfRelease", "Unknown"),
                    "Description": description,
                    # Optional: include a relevance hint for downstream evaluation
                    "Relevance_Score": f"{1 - dist:.3f}" if isinstance(dist, (int, float)) else "Unknown",
                })

        return str(formatted_results)

    except Exception as e:
        # Return a plain string to keep tool outputs simple for the agent
        return str([
            {
                "error": "Failed to retrieve from vector DB",
                "details": str(e),
            }
        ])

In [4]:
retrieve_game("When were Pokémon Gold and Silver released?")

'[{\'Platform\': \'Game Boy Color\', \'Name\': \'Pokémon Gold and Silver\', \'YearOfRelease\': 1999, \'Description\': \'Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.\', \'Relevance_Score\': \'0.774\'}, {\'Platform\': \'Game Boy Advance\', \'Name\': \'Pokémon Ruby and Sapphire\', \'YearOfRelease\': 2002, \'Description\': \'Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and double battles.\', \'Relevance_Score\': \'0.708\'}, {\'Platform\': \'Nintendo 64\', \'Name\': \'Super Mario 64\', \'YearOfRelease\': 1996, \'Description\': "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario\'s quest to rescue Princess Peach.", \'Relevance_Score\': \'0.544\'}, {\'Platform\': \'Super Nintendo Entertainment System (SNES)\', \'Name\': \'Super Mario World\', \'YearOfRelease\': 1990, \'Description\': \'A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land f

In [5]:
retrieve_game("Which was the first 3D platformer Mario game?")

'[{\'Platform\': \'Nintendo 64\', \'Name\': \'Super Mario 64\', \'YearOfRelease\': 1996, \'Description\': "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario\'s quest to rescue Princess Peach.", \'Relevance_Score\': \'0.782\'}, {\'Platform\': \'Super Nintendo Entertainment System (SNES)\', \'Name\': \'Super Mario World\', \'YearOfRelease\': 1990, \'Description\': \'A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser.\', \'Relevance_Score\': \'0.727\'}, {\'Platform\': \'Nintendo Switch\', \'Name\': \'Mario Kart 8 Deluxe\', \'YearOfRelease\': 2017, \'Description\': \'An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.\', \'Relevance_Score\': \'0.603\'}, {\'Platform\': \'GameCube\', \'Name\': \'Super Smash Bros. Melee\', \'YearOfRelease\': 2001, \'Description\': \'A crossover fighting game featuring characters from various Nintendo franchises battl

#### Evaluate Retrieval Tool

In [6]:
# Create evaluate_retrieval tool

from pydantic import BaseModel, Field
from lib.parsers import PydanticOutputParser

class EvaluationReport(BaseModel):
    """Evaluation report for retrieved documents"""
    useful: bool = Field(description="Whether the documents are useful to answer the question")
    description: str = Field(description="Detailed explanation about the evaluation result")

@tool
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    args:
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    try:
        # Initialize LLM for evaluation
        llm = LLM(model="gpt-4o-mini", temperature=0.1)

        # Build an instruction-focused evaluation prompt
        evaluation_prompt = f"""
        Your task is to evaluate if the documents are enough to respond the query.
        Give a detailed explanation, so it's possible to take an action to accept it or not.

        Evaluation criteria:
        - Relevance: Do the documents relate directly to the question?
        - Sufficiency: Is there enough detail to confidently answer?
        - Specificity: Are key details present (e.g., dates, platforms, names)?
        - Consistency: Do the documents agree or conflict?

        User Question:
        {question}

        Retrieved Documents (list of dicts or text):
        {retrieved_docs}

        Respond with JSON containing fields: useful (bool), description (string)
        """

        # Ask LLM to produce a structured response
        response = llm.invoke(
            input=evaluation_prompt,
            response_format=EvaluationReport,
        )

        # Parse the structured response
        parser = PydanticOutputParser(model_class=EvaluationReport)
        evaluation = parser.parse(response)

        return f"Useful: {evaluation.useful}, Description: {evaluation.description}"

    except Exception as e:
        # Fallback heuristic if LLM or parsing fails
        docs_text = str(retrieved_docs).lower()
        question_text = question.lower()

        key_terms = [w for w in question_text.replace("?", "").split() if len(w) > 3]
        matches = sum(1 for term in key_terms if term in docs_text)

        useful = matches > 0 and len(docs_text) > 50
        description = (
            f"Fallback evaluation: Found {matches} matching key terms in retrieved docs. "
            f"Original error: {str(e)}"
        )
        return f"Useful: {useful}, Description: {description}"

In [7]:
# Quick test: retrieve and evaluate
query = "When was Pokémon Gold and Silver released?"
retrieved = retrieve_game(query)
evaluate_retrieval(query, retrieved)

"Useful: True, Description: The retrieved documents contain relevant information regarding the release of Pokémon Gold and Silver. Specifically, one document directly addresses the user question by stating that Pokémon Gold and Silver were released in 1999 on the Game Boy Color. This document is highly relevant, as it provides the exact name of the game, the platform it was released on, and the year of release, which are all key details necessary to answer the question confidently. \n\nThe other documents retrieved discuss different Pokémon games and platforms, but they do not pertain to the user's question about Pokémon Gold and Silver. However, since the first document provides a clear and direct answer, the overall evaluation is that the documents are sufficient to respond to the query. \n\nIn summary, the relevant document provides the necessary details (name, platform, year) and there are no conflicting details present, making the information consistent and reliable."

#### Game Web Search Tool

In [8]:
# Create game_web_search tool

from tavily import TavilyClient

@tool
def game_web_search(question: str) -> str:
    """
    Web search: Searches the internet for information about the gaming industry
    args:
    - question: a question about game industry.
    """
    try:
        client = TavilyClient(api_key=TAVILY_API_KEY)

        # Bias the query toward gaming context
        query = f"video games gaming industry: {question}"
        results = client.search(
            query=query,
            search_depth="advanced",
            max_results=5,
            include_answer=True,
            include_raw_content=False,
        )

        formatted = []
        if isinstance(results, dict):
            if results.get("answer"):
                formatted.append({"Summary": results["answer"]})
            for i, item in enumerate((results.get("results") or [])[:5], start=1):
                formatted.append({
                    "Title": item.get("title", ""),
                    "URL": item.get("url", ""),
                    "Content": item.get("content", ""),
                    "Score": item.get("score", 0),
                })
        else:
            formatted.append({"info": "Unexpected response shape from Tavily"})

        return str(formatted) if formatted else "[]"

    except Exception as e:
        return str([
            {
                "error": "Web search failed",
                "details": str(e),
                "hint": "Check TAVILY_API_KEY and network connectivity",
            }
        ])

In [9]:
# Quick test: web search fallback
# Example: a question likely requiring web info
game_web_search("What is the release date of Metroid Prime 4?")

'[{\'Summary\': \'Metroid Prime 4: Beyond is set for release on December 4, 2025. It will be available on Nintendo Switch and Switch 2.\'}, {\'Title\': \'Metroid Prime 4: Beyond gets a new trailer and a December release ...\', \'URL\': \'https://nintendowire.com/news/2025/09/17/metroid-prime-4-beyond-gets-a-new-trailer-and-a-december-release-date/\', \'Content\': \'Of course, that wasn’t all that was shown off here. Among the rest of the gameplay footage and a brief new look at the returning Sylux, we got some exciting release date news. Metroid Prime 4: Beyond is set for release on both Nintendo Switch and Nintendo Switch 2 on December 4th, 2025. In addition, an artbook for the game is coming out on October 28th. That’s not even the end of it though, as to wrap it all up we got confirmation of three new amiibo for the game to release alongside it! [...] For all of those out there who have been eagerly awaiting the release of Metroid Prime 4, it’s finally almost here. Now with a known 

### Agent

In [10]:
# Create your Agent abstraction using StateMachine

# Instantiate the UdaPlay Research Agent
udaplay_agent = Agent(
    model_name="gpt-4o-mini",
    temperature=0.7,
    instructions="""
You are UdaPlay, an AI Research Agent for the video game industry.

Capabilities:
1. Answer questions using internal knowledge from the gaming database (RAG)
2. When internal info is insufficient, search the web
3. Maintain conversation state across turns
4. Return structured, clear, and accurate outputs
5. Be transparent about sources and uncertainty

Workflow:
1. First use the tool `retrieve_game` to search the internal vector database
2. Use `evaluate_retrieval` to decide if the retrieved info is sufficient
3. If not sufficient, use `game_web_search` to gather more info
4. Synthesize a final answer, citing where information came from (internal DB and/or web)

Guidelines:
- Be precise and factual: include relevant details like release dates and platforms
- Prefer evidence over speculation; when unsure, explicitly indicate uncertainty
- Keep answers concise but complete; use bullet points when useful
- Never fabricate citations; only cite the sources you truly used
""",
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
)

print("UdaPlay Agent created! Model:", udaplay_agent.model_name)
print("Tools:", ", ".join(t.name for t in udaplay_agent.tools))

UdaPlay Agent created! Model: gpt-4o-mini
Tools: retrieve_game, evaluate_retrieval, game_web_search


In [11]:
# Invoke your agent on sample questions

print("=== UdaPlay Agent Quick Run ===\n")

samples = [
    "When was Pokémon Gold and Silver released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for PlayStation 5?",
]

for i, q in enumerate(samples, 1):
    print(f"Q{i}: {q}")
    try:
        run = udaplay_agent.invoke(q)
        final_state = run.get_final_state()
        if final_state and final_state.get("messages"):
            ai_msgs = [m for m in final_state["messages"] if hasattr(m, 'content') and m.content]
            if ai_msgs:
                print("Answer:")
                print(ai_msgs[-1].content)
            else:
                print("No AI message content available.")
        else:
            print("No final state/messages.")
    except Exception as e:
        print("Error:", e)
    print("-" * 60)

print("Done.")

=== UdaPlay Agent Quick Run ===

Q1: When was Pokémon Gold and Silver released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Answer:
Pokémon Gold and Silver were released in **1999** for the **Game Boy Color**. These games introduced the second generation of Pokémon, along with new regions and gameplay mechanics.
------------------------------------------------------------
Q2: Which one was the first 3D platformer Mario game?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMa

### (Optional) Advanced

In [12]:
# Advanced: Long-Term Memory + Explicit State Machine with Tool Nodes

from typing import TypedDict, Optional, Dict, List
from datetime import datetime, timedelta
import json

from lib.memory import LongTermMemory, MemoryFragment, TimestampFilter
from lib.vector_db import VectorStoreManager
from lib.state_machine import StateMachine, Step, EntryPoint, Termination
from lib.llm import LLM
from lib.tooling import Tool, tool

# 1) Initialize Long-Term Memory (LTM)
#    Uses OpenAI embeddings via VectorStoreManager; persists inside a Chroma collection.
vector_manager = VectorStoreManager(OPENAI_API_KEY)
long_memory = LongTermMemory(vector_manager)

# 2) Create memory functions that will be wrapped as tools
@tool
def save_memory_ltm(owner: str, content: str, namespace: str = "default") -> str:
    """
    Save a memory fragment for a given `owner` into long-term memory.
    args:
    - owner: user id or session id
    - content: the fact/preference/answer to retain
    - namespace: logical grouping of memories
    """
    try:
        long_memory.register(
            MemoryFragment(content=content, owner=owner, namespace=namespace)
        )
        return json.dumps({"status": "ok", "saved": True})
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

@tool
def recall_memory_ltm(owner: str, query: str, namespace: str = "default", limit: int = 3, within_hours: Optional[int] = None) -> str:
    """
    Recall relevant memories for `owner` semantically similar to `query`.
    args:
    - owner: user id or session id
    - query: search text to match memories
    - namespace: filter namespace
    - limit: number of memories to return
    - within_hours: if provided, only return memories newer than now - within_hours
    """
    try:
        ts_filter = None
        if within_hours is not None:
            cutoff = int((datetime.now() - timedelta(hours=within_hours)).timestamp())
            ts_filter = TimestampFilter(greater_than_value=cutoff)
        result = long_memory.search(query_text=query, owner=owner, limit=limit, timestamp_filter=ts_filter, namespace=namespace)
        out = []
        for frag, dist in zip(result.fragments, result.metadata.get("distances", [])):
            out.append({
                "content": frag.content,
                "owner": frag.owner,
                "namespace": frag.namespace,
                "timestamp": frag.timestamp,
                "distance": dist,
            })
        return json.dumps(out, ensure_ascii=False)
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

# 3) Create other tool functions that we'll use directly in StateMachine

@tool
def retrieve_game_ltm(query: str) -> str:
    """
    Semantic search: Finds most results in the vector DB
    """
    try:
        import chromadb
        from chromadb.utils import embedding_functions
        
        chroma_client = chromadb.PersistentClient(path="chromadb")
        embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY)
        collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)
        results = collection.query(
            query_texts=[query],
            n_results=5,
            include=["documents", "metadatas", "distances"],
        )
        formatted_results = []
        docs = results.get("documents") or []
        metas = results.get("metadatas") or []
        dists = results.get("distances") or []
        if docs and len(docs) > 0:
            for i, doc in enumerate(docs[0]):
                meta = metas[0][i] if metas and metas[0] and i < len(metas[0]) else {}
                dist = dists[0][i] if dists and dists[0] and i < len(dists[0]) else None
                description = meta.get("Description") or doc
                formatted_results.append({
                    "Platform": meta.get("Platform", "Unknown"),
                    "Name": meta.get("Name", "Unknown"),
                    "YearOfRelease": meta.get("YearOfRelease", "Unknown"),
                    "Description": description,
                    "Relevance_Score": f"{1 - dist:.3f}" if isinstance(dist, (int, float)) else "Unknown",
                })
        return str(formatted_results)
    except Exception as e:
        return str([{ "error": "Failed to retrieve from vector DB", "details": str(e) }])

@tool
def evaluate_retrieval_ltm(question: str, retrieved_docs: str) -> str:
    """
    Evaluate if retrieved documents are useful for answering the question
    """
    try:
        from pydantic import BaseModel, Field
        from lib.parsers import PydanticOutputParser

        class EvaluationReport(BaseModel):
            useful: bool = Field(description="Whether the documents are useful to answer the question")
            description: str = Field(description="Detailed explanation about the evaluation result")

        llm = LLM(model="gpt-4o-mini", temperature=0.1)
        evaluation_prompt = f"""
Your task is to evaluate if the documents are enough to respond the query.
Give a detailed explanation, so it's possible to take an action to accept it or not.

Evaluation criteria:
- Relevance: Do the documents relate directly to the question?
- Sufficiency: Is there enough detail to confidently answer?
- Specificity: Are key details present (e.g., dates, platforms, names)?
- Consistency: Do the documents agree or conflict?

User Question:
{question}

Retrieved Documents (list of dicts or text):
{retrieved_docs}

Respond with JSON containing fields: useful (bool), description (string)
"""
        response = llm.invoke(input=evaluation_prompt, response_format=EvaluationReport)
        parser = PydanticOutputParser(model_class=EvaluationReport)
        evaluation = parser.parse(response)
        return f"Useful: {evaluation.useful}, Description: {evaluation.description}"
    except Exception as e:
        docs_text = str(retrieved_docs).lower()
        question_text = question.lower()
        key_terms = [w for w in question_text.replace("?", "").split() if len(w) > 3]
        matches = sum(1 for term in key_terms if term in docs_text)
        useful = matches > 0 and len(docs_text) > 50
        description = (
            f"Fallback evaluation: Found {matches} matching key terms in retrieved docs. Original error: {str(e)}"
        )
        return f"Useful: {useful}, Description: {description}"

@tool
def game_web_search_ltm(question: str) -> str:
    """
    Web search for gaming industry information
    """
    try:
        from tavily import TavilyClient
        
        client = TavilyClient(api_key=TAVILY_API_KEY)
        query = f"video games gaming industry: {question}"
        results = client.search(
            query=query,
            search_depth="advanced",
            max_results=5,
            include_answer=True,
            include_raw_content=False,
        )
        formatted = []
        if isinstance(results, dict):
            if results.get("answer"):
                formatted.append({"Summary": results["answer"]})
            for i, item in enumerate((results.get("results") or [])[:5], start=1):
                formatted.append({
                    "Title": item.get("title", ""),
                    "URL": item.get("url", ""),
                    "Content": item.get("content", ""),
                    "Score": item.get("score", 0),
                })
        else:
            formatted.append({"info": "Unexpected response shape from Tavily"})
        return str(formatted) if formatted else "[]"
    except Exception as e:
        return str([{ "error": "Web search failed", "details": str(e), "hint": "Check TAVILY_API_KEY and network connectivity" }])

# Verify all tools have proper names
tools_for_agent = [retrieve_game_ltm, evaluate_retrieval_ltm, game_web_search_ltm, recall_memory_ltm, save_memory_ltm]
for t in tools_for_agent:
    if not hasattr(t, 'name') or not t.name:
        print(f"Warning: Tool {t} doesn't have a proper name")
    else:
        print(f"Tool {t.name} is properly configured")

# 4) Update the Agent to include long-term memory tools (v2)
udaplay_agent_v2 = Agent(
    model_name="gpt-4o-mini",
    temperature=0.7,
    instructions=
    """
You are UdaPlay, an AI Research Agent for the video game industry.
In addition to RAG and web search, you can save and recall useful user memories.
When you reach a confident and useful answer, store a brief fact using the `save_memory_ltm` tool.
Before answering questions, try to use `recall_memory_ltm` to retrieve relevant context.
    """.strip(),
    tools=tools_for_agent,
)

print(f"UdaPlay Agent v2 created with {len(udaplay_agent_v2.tools)} tools")

# 5) Build an explicit StateMachine where tools are predefined nodes

# State schema for the explicit research workflow
class ResearchState(TypedDict, total=False):
    question: str
    owner: str
    session_id: str
    memory_results: str
    retrieved_docs: str
    retrieval_useful: bool
    evaluation_notes: str
    web_results: str
    final_answer: str

# Helper: parse the evaluate_retrieval string result

def _parse_evaluation(text: str) -> Dict[str, Optional[str]]:
    # Expected like: "Useful: True, Description: ..."
    useful = None
    description = None
    try:
        lower = text.strip()
        parts = lower.split(",", 1)
        if parts:
            if "true" in parts[0].lower():
                useful = True
            elif "false" in parts[0].lower():
                useful = False
        if len(parts) > 1:
            desc = parts[1].strip()
            if desc.lower().startswith("description:"):
                desc = desc[len("description:"):].strip()
            description = desc
    except Exception:
        pass
    return {"useful": useful, "description": description}

# Steps definitions - Now using the function directly

# Recall memory step
recall_step = Step[ResearchState](
    "recall_memory",
    lambda state: {
        "memory_results": recall_memory_ltm.func(
            owner=state.get("owner", state.get("session_id", "default")),
            query=state["question"],
            namespace="default",
            limit=3,
            within_hours=None,
        )
    }
)

# Retrieve from internal DB step
retrieve_step = Step[ResearchState](
    "retrieve_game",
    lambda state: {
        "retrieved_docs": retrieve_game_ltm.func(query=state["question"])
    }
)

# Evaluate retrieval utility step

def _eval_logic(state: ResearchState) -> ResearchState:
    raw = evaluate_retrieval_ltm.func(
        question=state["question"],
        retrieved_docs=state.get("retrieved_docs", "")
    )
    parsed = _parse_evaluation(raw)
    return {
        "retrieval_useful": bool(parsed.get("useful", False)),
        "evaluation_notes": parsed.get("description", "")
    }

evaluate_step = Step[ResearchState]("evaluate_retrieval", _eval_logic)

# Web search step (fallback)
web_search_step = Step[ResearchState](
    "game_web_search",
    lambda state: {
        "web_results": game_web_search_ltm.func(question=state["question"])
    }
)

# Synthesis step via LLM

def _synthesize_logic(state: ResearchState) -> ResearchState:
    llm = LLM(model="gpt-4o-mini", temperature=0.3)
    context_blobs: List[str] = []
    if state.get("memory_results"):
        context_blobs.append(f"Recalled Memory: {state['memory_results']}")
    if state.get("retrieved_docs"):
        context_blobs.append(f"Internal DB: {state['retrieved_docs']}")
    if state.get("web_results"):
        context_blobs.append(f"Web: {state['web_results']}")

    context_str = "\n\n".join(context_blobs) if context_blobs else "(no extra context)"
    prompt = f"""
You are UdaPlay. Answer the gaming question using the provided context.
- Cite whether the info came from internal DB, web, and/or memory.
- Be precise; include dates/platforms where relevant.
- If uncertain, say so.

Question:
{state['question']}

Context:
{context_str}

Provide a clear, concise answer.
""".strip()
    ai = llm.invoke(prompt)
    return {"final_answer": ai.content}

synthesize_step = Step[ResearchState]("synthesize_answer", _synthesize_logic)

# Store final answer into LTM step

def _store_answer_logic(state: ResearchState) -> ResearchState:
    owner = state.get("owner", state.get("session_id", "default"))
    summary = f"Q: {state['question']}\nA: {state.get('final_answer','')}"
    _ = save_memory_ltm.func(owner=owner, content=summary, namespace="answers")
    return {}

store_answer_step = Step[ResearchState]("store_answer", _store_answer_logic)

# Build the machine and transitions
research_machine = StateMachine[ResearchState](ResearchState)
entry = EntryPoint[ResearchState]()
term = Termination[ResearchState]()

research_machine.add_steps([
    entry,
    recall_step,
    retrieve_step,
    evaluate_step,
    web_search_step,
    synthesize_step,
    store_answer_step,
    term,
])

# Flow: entry -> recall -> retrieve -> evaluate -> (useful? synthesize : web_search -> synthesize) -> store -> term
research_machine.connect(entry, recall_step)
research_machine.connect(recall_step, retrieve_step)
research_machine.connect(retrieve_step, evaluate_step)

# Branch based on evaluation

def _route_after_eval(state: ResearchState):
    return synthesize_step if state.get("retrieval_useful") else web_search_step

research_machine.connect(evaluate_step, [synthesize_step, web_search_step], _route_after_eval)
# If web search happened, go to synthesize next
research_machine.connect(web_search_step, synthesize_step)
# After synthesize, store answer, then terminate
research_machine.connect(synthesize_step, store_answer_step)
research_machine.connect(store_answer_step, term)

print("Explicit Research StateMachine constructed with tool nodes.")

# 6) Quick demo run (single question). Feel free to adapt/loop as needed.
owner_id = "default_user"
question = "When was Pokémon Gold and Silver released?"
run = research_machine.run({
    "question": question,
    "owner": owner_id,
    "session_id": "default",
})
final_state = run.get_final_state()
print("Final Answer:\n", (final_state or {}).get("final_answer", "<no answer>"))

Tool retrieve_game_ltm is properly configured
Tool evaluate_retrieval_ltm is properly configured
Tool game_web_search_ltm is properly configured
Tool recall_memory_ltm is properly configured
Tool save_memory_ltm is properly configured
UdaPlay Agent v2 created with 5 tools
Explicit Research StateMachine constructed with tool nodes.
[StateMachine] Starting: __entry__
[StateMachine] Executing step: recall_memory
[StateMachine] Executing step: retrieve_game
[StateMachine] Executing step: evaluate_retrieval
[StateMachine] Executing step: synthesize_answer
[StateMachine] Executing step: store_answer
[StateMachine] Terminating: __termination__
Final Answer:
 Pokémon Gold and Silver were released in 1999 for the Game Boy Color. This information is sourced from the internal database.


In [14]:
# Test the enhanced agent with memory capabilities and verify the StateMachine

print("=== Testing UdaPlay Agent v2 with Memory ===\n")

test_questions = [
    "When was Pokémon Gold and Silver released?",
    "What do you remember about Pokémon games?",  # This should recall the previous answer
]

for i, q in enumerate(test_questions, 1):
    print(f"Q{i}: {q}")
    try:
        run = udaplay_agent_v2.invoke(q)
        final_state = run.get_final_state()
        if final_state and final_state.get("messages"):
            ai_msgs = [m for m in final_state["messages"] if hasattr(m, 'content') and m.content]
            if ai_msgs:
                print("Answer:")
                print(ai_msgs[-1].content[:500] + "..." if len(ai_msgs[-1].content) > 500 else ai_msgs[-1].content)
            else:
                print("No AI message content available.")
        else:
            print("No final state/messages.")
    except Exception as e:
        print("Error:", e)
    print("-" * 60)

print("Done.\n")

# Test the explicit StateMachine with a different question
print("=== Testing Explicit StateMachine with Web Search Fallback ===\n")

# Ask about a game that might not be in our vector DB
owner_id = "test_user_2"
question = "What is the latest information about Elden Ring DLC?"
run = research_machine.run({
    "question": question,
    "owner": owner_id,
    "session_id": "session_2",
})
final_state = run.get_final_state()
print("Question:", question)
print("Final Answer:\n", (final_state or {}).get("final_answer", "<no answer>"))
print("\nState Machine Execution Flow:")
print(f"- Memory Results: {'Found' if final_state.get('memory_results') else 'None'}")
print(f"- Retrieved Docs: {'Found' if final_state.get('retrieved_docs') else 'None'}")
print(f"- Retrieval Useful: {final_state.get('retrieval_useful', 'N/A')}")
print(f"- Web Results: {'Found' if final_state.get('web_results') else 'None'}")


=== Testing UdaPlay Agent v2 with Memory ===

Q1: When was Pokémon Gold and Silver released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Answer:
Pokémon Gold and Silver were released in Japan on **November 21, 1999**, in North America on **October 15, 2000**, and in Europe on **April 6, 2001**. 

Would you like to learn more about these games or any specific features?
------------------------------------------------------------
Q2: What do you remember about Pokémon games?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Answer:
I still don't have any specific memories about Pokémon games stored. However, I can share general information or answer questions yo